In [1]:
import os
import re
import requests
import time
import json
import logging
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from transformers import AutoTokenizer
import nltk

c:\Users\thain\anaconda3\envs\ragbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('punkt', quiet=True)

# Cấu hình logging
logging.basicConfig(
    filename="crawl.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

API_URL = "https://vi.wikipedia.org/w/api.php"
HEADERS = {"User-Agent": "RagBot/1.0"}
CHECKPOINT_FILE = "checkpoint.json"
OUTPUT_FOLDER = "wiki_chunks"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

category_whitelist = [
    "Việt Nam",
    "Lịch sử Việt Nam",
    "Địa lý Việt Nam",
    "Khởi nghĩa Việt Nam",
    "Trận đánh liên quan tới Việt Nam",
    "Triều đại Việt Nam",
    "Quân chủ Việt Nam",
    "Đảng Cộng sản Việt Nam",
    "Chiến tranh Việt Nam",
    "Anh hùng dân tộc Việt Nam",
    "Kỷ nguyên vươn mình"
]

# Load checkpoint
if os.path.exists(CHECKPOINT_FILE):
    with open(CHECKPOINT_FILE, "r", encoding="utf-8") as f:
        visited_pages = set(json.load(f))
else:
    visited_pages = set()

def save_checkpoint():
    with open(CHECKPOINT_FILE, "w", encoding="utf-8") as f:
        json.dump(list(visited_pages), f, ensure_ascii=False, indent=2)

def request_with_retry(params, max_retries=5, timeout=20):
    for attempt in range(max_retries):
        try:
            response = requests.get(API_URL, headers=HEADERS, params=params, timeout=timeout)
            response.raise_for_status()
            time.sleep(0.5)  # Giảm tốc độ gửi yêu cầu
            return response.json()
        except Exception as e:
            logging.error(f"[Retry {attempt+1}] Error: {e}")
            print(f"[Retry {attempt+1}] Error: {e}")
            time.sleep(2)
    return None

In [3]:
def get_category_members(category, cmcontinue=None):
    params = {
        "action": "query",
        "format": "json",
        "list": "categorymembers",
        "cmtitle": f"Category:{category}",
        "cmlimit": "max",
        "cmtype": "page|subcat"
    }
    if cmcontinue:
        params["cmcontinue"] = cmcontinue

    data = request_with_retry(params)
    if not data:
        return [], None
    members = data["query"]["categorymembers"]
    next_continue = data.get("continue", {}).get("cmcontinue")
    return members, next_continue

def get_page_content(pageid):
    params = {
        "action": "query",
        "format": "json",
        "pageids": pageid,
        "prop": "extracts",
        "explaintext": 1
    }
    data = request_with_retry(params)
    if not data:
        return ""
    pages = data["query"]["pages"]
    return list(pages.values())[0].get("extract", "")

def get_multiple_page_contents(pageids):
    with ThreadPoolExecutor(max_workers=30) as executor: # Tối đa 30 luồng
        contents = list(executor.map(get_page_content, pageids))
    return contents

def sanitize_filename(filename):
    return re.sub(r'[\\/*?:"<>|]', "_", filename)

def clean_wiki_content(content):
    content = re.sub(r'==\s*(Tham khảo|Liên kết ngoài|Chú thích|Xem thêm)\s*==.*', '', content, flags=re.DOTALL)
    content = re.sub(r'\n\s*\n', '\n', content)
    content = re.sub(r'\[\[\w+:\w+\]\]', '', content)
    content = content.strip()
    return content

In [4]:
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-base")

def chunk_text_by_sentences(text, max_tokens=512):
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_tokens = 0
    
    for sentence in sentences:
        tokens = len(tokenizer.encode(sentence, add_special_tokens=False))
        if current_tokens + tokens > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            current_tokens = tokens
        else:
            current_chunk.append(sentence)
            current_tokens += tokens
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks

def chunk_by_sections(text, max_tokens=512):
    try:
        text = clean_wiki_content(text)
        sections = re.split(r'(?m)^==\s*[^=]+\s*==\s*\n', text)
        section_titles = re.findall(r'(?m)^==\s*([^=]+)\s*==\s*\n', text)
        
        if not section_titles:
            section_titles = ["Giới thiệu"]
            sections = [text] if text else []
        else:
            if sections[0].strip() == "":
                sections.pop(0)
            else:
                section_titles.insert(0, "Giới thiệu")
        
        chunks = []
        for title, section in zip(section_titles, sections):
            section = section.strip()
            if not section:
                continue
            section_content = f"{title.strip()}\n{section}"
            # Ước lượng số token (thay vì encode toàn bộ)
            estimated_tokens = len(section_content.split()) * 1.5  # 1 từ ~ 1.5 token
            if estimated_tokens <= max_tokens:
                # Kiểm tra token thực tế
                tokens = tokenizer.encode(section_content, add_special_tokens=False)
                if len(tokens) <= max_tokens:
                    chunks.append(section_content)
                else:
                    sub_chunks = chunk_text_by_sentences(section_content, max_tokens)
                    chunks.extend(sub_chunks)
            else:
                # Section dài, chia nhỏ ngay
                sub_chunks = chunk_text_by_sentences(section_content, max_tokens)
                chunks.extend(sub_chunks)
        
        return chunks
    except Exception as e:
        logging.error(f"Lỗi khi chunk bài viết: {e}")
        print(f"  ⚠️ Lỗi chunking: {e}. Bỏ qua bài viết.")
        return []

c:\Users\thain\anaconda3\envs\ragbot\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
def save_to_jsonl(data, category):
    jsonl_file = f"{OUTPUT_FOLDER}/{sanitize_filename(category)}.jsonl"
    with open(jsonl_file, "a", encoding="utf-8") as f:
        for entry in data:
            json.dump(entry, f, ensure_ascii=False)
            f.write("\n")

def save_page_content(title, content, category):
    if title in visited_pages:
        logging.info(f"Bỏ qua {title} vì đã được xử lý")
        print(f"  ⚠️ Bỏ qua {title} vì đã được xử lý")
        return
    chunks = chunk_by_sections(content)
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    data_to_save = []

    for idx, chunk in enumerate(chunks):
        token_count = len(tokenizer.encode(chunk, add_special_tokens=False))
        logging.info(f"Chunk {idx} of {title}: {token_count} tokens")
        metadata = {
            "title": title,
            "category": category,
            "timestamp": timestamp,
            "chunk_id": idx,
            "total_chunks": len(chunks),
            "url": f"https://vi.wikipedia.org/wiki/{title.replace(' ', '_')}"
        }
        data_to_save.append({
            "text": chunk,
            "metadata": metadata
        })
    
    save_to_jsonl(data_to_save, category)
    visited_pages.add(title)
    save_checkpoint()
    logging.info(f"Đã lưu {title} với {len(chunks)} chunks")

In [6]:
def is_relevant_category(category):
    irrelevant_keywords = [
        "bản mẫu", "cảnh sát", "hoa khôi", "tổ chức", "y tế", "hình ảnh", "sơ khai",
        "pháp lý", "nháp", "thời trang"
    ]
    # Trả về False nếu danh mục chứa từ khóa không liên quan
    return not any(keyword in category.lower() for keyword in irrelevant_keywords)

def crawl_category(category, depth=2):
    print(f"\n🌀 Crawling category: {category} (depth={depth})")
    if not is_relevant_category(category):
        logging.info(f"Bỏ qua danh mục không liên quan: {category}")
        print(f"  ⚠️ Bỏ qua danh mục không liên quan: {category}")
        return 0
    # Giảm độ sâu cho các danh mục tiềm năng không liên quan
    if any(keyword in category.lower() for keyword in ["văn hóa", "chính trị", "kinh tế", "xã hội", "giáo dục", "du lịch"]):
        depth = min(depth, 1)  # Giới hạn độ sâu
        logging.info(f"Giới hạn độ sâu crawl cho {category} xuống {depth}")
    to_crawl = [(category, depth)]
    page_count = 0

    while to_crawl:
        current_cat, current_depth = to_crawl.pop()
        print(f"- Đang xử lý: {current_cat} (depth={current_depth})")
        cmcontinue = None
        while True:
            members, cmcontinue = get_category_members(current_cat, cmcontinue)
            pageids = [m["pageid"] for m in members if m["ns"] == 0 and m["title"] not in visited_pages]
            subcats = [m["title"].split(":", 1)[-1] for m in members if m["ns"] == 14 and current_depth > 0]

            if pageids:
                contents = get_multiple_page_contents(pageids)
                for member, content in zip([m for m in members if m["ns"] == 0], contents):
                    title = member["title"]
                    # Bỏ qua các trang không liên quan
                    irrelevant_page_keywords = [
                        "nháp", "atradius", "gala", "nội dung tự do", "quan hệ",
                        "thời trang", "pháp lý", "kinh tế mới", "xã hội dân sự"
                    ]
                    if any(keyword in title.lower() for keyword in irrelevant_page_keywords):
                        logging.info(f"Bỏ qua trang không liên quan: {title}")
                        print(f"  ⚠️ Bỏ qua trang không liên quan: {title}")
                        visited_pages.add(title)
                        save_checkpoint()
                        continue
                    # Kiểm tra nội dung trang có từ khóa lịch sử/địa lý
                    relevant_content_keywords = [
                        # Lịch sử - Chung
                        "lịch sử", "thế kỷ", "thời kỳ", "kỷ nguyên", "thời", "sự kiện",
                        "cách mạng", "độc lập", "thống nhất", "biến cố", "tấn công",
                        # Triều đại và vua chúa
                        "triều đại", "vua", "hoàng", "chúa", "kinh thành", "cố đô",
                        # Chiến tranh và khởi nghĩa
                        "chiến tranh", "khởi nghĩa", "trận đánh", "quân đội", "sứ quân", "kháng chiến",
                        # Nhân vật lịch sử
                        "anh hùng", "tướng", "nhân vật", "nhà",  # Nhà thơ, nhà văn lịch sử
                        # Địa lý - Chung
                        "địa lý", "địa danh", "lãnh thổ", "tỉnh", "huyện", "thành phố",
                        # Địa lý tự nhiên
                        "sông", "núi", "đồng bằng", "vịnh", "hồ", "biển", "đảo",
                        # Văn hóa lịch sử và di sản
                        "văn minh", "di sản", "đền", "chùa", "thành", "lăng", "bảo vật", "truyền thống", "dân ca",
                        "văn học", "nhà thơ", "lễ hội",
                        # Xã hội lịch sử
                        "làng xã", "cộng đồng", "phong kiến", "xã hội", "thời đại",
                        # Dân tộc và vùng
                        "dân tộc", "vùng cao", "vùng", "người"
                    ]
                    if content and not any(keyword in content.lower() for keyword in relevant_content_keywords):
                        logging.info(f"Bỏ qua trang không có nội dung lịch sử/địa lý: {title}")
                        print(f"  ⚠️ Bỏ qua trang không có nội dung lịch sử/địa lý: {title}")
                        visited_pages.add(title)
                        save_checkpoint()
                        continue
                    if content and title not in visited_pages:
                        print(f"  📄 {title}")
                        save_page_content(title, content, current_cat)
                        page_count += 1

            to_crawl.extend((subcat, current_depth - 1) for subcat in subcats)
            if not cmcontinue:
                break
    return page_count

# Crawl all categories
total_pages = 0
depth = 2
for category in category_whitelist:
    pages_crawled = crawl_category(category, depth)
    total_pages += pages_crawled

print(f"\n🎉 Đã lưu {total_pages} bài viết")


🌀 Crawling category: Việt Nam (depth=2)
- Đang xử lý: Việt Nam (depth=2)
- Đang xử lý: Sơ khai Việt Nam (depth=1)
- Đang xử lý: Bản mẫu sơ khai Việt Nam (depth=0)
- Đang xử lý: Sơ khai văn hóa Việt Nam (depth=0)
- Đang xử lý: Sơ khai truyền thông đại chúng Việt Nam (depth=0)
- Đang xử lý: Sơ khai tôn giáo Việt Nam (depth=0)
- Đang xử lý: Sơ khai tổ chức Việt Nam (depth=0)
- Đang xử lý: Sơ khai thể thao Việt Nam (depth=0)
- Đang xử lý: Sơ khai nhân vật Việt Nam (depth=0)
- Đang xử lý: Sơ khai lịch sử Việt Nam (depth=0)
- Đang xử lý: Sơ khai kiến trúc Việt Nam (depth=0)
- Đang xử lý: Sơ khai hành chính Việt Nam (depth=0)
- Đang xử lý: Sơ khai giao thông Việt Nam (depth=0)
- Đang xử lý: Sơ khai địa lý Việt Nam (depth=0)
- Đang xử lý: Sơ khai điện lực Việt Nam (depth=0)
- Đang xử lý: Sơ khai công ty Việt Nam (depth=0)
- Đang xử lý: Sơ khai công trình xây dựng Việt Nam (depth=0)
- Đang xử lý: Sơ khai Chiến tranh Việt Nam (depth=0)
- Đang xử lý: Sơ khai ẩm thực Việt Nam (depth=0)
- Đang xử 

Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors


  📄 Đồng bằng sông Cửu Long
  📄 Danh sách điểm cực trị của Việt Nam
  📄 Đồng bằng sông Hồng
- Đang xử lý: Địa danh Việt Nam (depth=0)
- Đang xử lý: Địa chất Việt Nam (depth=0)
  📄 Bồn trũng Sông Hồng
  📄 Đới đứt gãy Sông Hồng
  📄 Henri Fontaine
  📄 Josué Hoffet
  📄 Jacques Deprat
  📄 Honoré Lantenois
  📄 Henri Mansuy
  📄 Suối nước khoáng Bang
  📄 Tổng cục Địa chất và Khoáng sản Việt Nam
- Đang xử lý: Công viên Việt Nam (depth=0)
  📄 Công viên 29 tháng 3
  📄 Công viên Lê-nin
  📄 Công viên Hội An (Quảng Nam)
- Đang xử lý: Cảng Việt Nam (depth=0)
  📄 Cảng Cam Ranh
  📄 Bến Nhà Rồng
  📄 Bến Thủy
  📄 Cảng Cái Lân
  📄 Vịnh Cam Ranh
  📄 Cảng An Thới
  📄 Cảng biển Ninh Bình
  📄 Cảng Diêm Điền
  📄 Cảng Nha Trang
  📄 Cảng Tân Cảng - Cái Mép
  📄 Cửa biển Mỹ Á
  📄 Cảng biển nước sâu Cửa Lò
  📄 Cảng Cửa Việt
  📄 Cảng Dung Quất
  📄 Cảng Đà Nẵng
  📄 Cảng Đình Vũ
  📄 Cảng Hải Phòng
  📄 Cảng Hòn Gai
  📄 Cảng Kỳ Hà
  📄 Cảng Nghi Sơn
  📄 Cảng Ninh Phúc
  📄 Cảng Quy Nhơn
  📄 Sa Kỳ
- Đang xử lý: Biên giới V